In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.special import kl_div

import tensorflow as tf 
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
import subprocess
import random
import time 

from utils import *

%matplotlib inline
%reload_ext autoreload
%autoreload 2

2024-06-11 14:34:52.816734: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 14:34:53.642237: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from platform import python_version
python_version()

'3.8.10'

# Section 1: Helper functions and set-ups 

In [3]:
## Graphing set-up 
import seaborn as sns
x = np.linspace(-4, 4, 100)
tencent_blue = (0,0.3215686274509804,0.8509803921568627)
tencent_orange = (0.9333333333333333, 0.49411764705882355, 0.2784313725490196)

# Calculate y-values for the standard normal density curve
y_standard_normal = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * x**2)

# Section 3: Simultation 

In [4]:
## Modifying the tensor for 3d input 
class MyModel_multiple(Model):
    def __init__(self, k, num_treats):
        super(MyModel_multiple, self).__init__()
        self.k = k
        self.num_treats = num_treats
        self.groupNames = ['A'] + ['B' + str(i+1) for i in range(self.num_treats)]
        self.baseline_logit = Dense(1, activation = "linear")
        self.outcome = Dense(1, activation = "linear")
        self.logit_dense_layer = {} 
        for g in self.groupNames:
            self.logit_dense_layer[g] = Dense(1, activation = "linear")
        self.softmax = tf.keras.activations.softmax
        
    
    def call(self, inputs):

        split_structure =  [2] + [1] * self.num_treats + [1]
        splitted_elements = tf.split(inputs, split_structure, axis=2)
        x1 = splitted_elements[0]
        exposure = tf.squeeze(splitted_elements[self.num_treats + 1], axis=-1)
        _, K, dim_x = x1.shape
        
        
        ## Step 1: Reshape the input 
        reshape_x1 = tf.reshape(x1, (-1, dim_x))
        
        ## Step 2: Score 
        ### Baseline logit
        x1_final = self.baseline_logit(x1)
        
        ### Uplift
        for i in range(self.num_treats):
            w_g = splitted_elements[i + 1]
            xg_hidden = self.logit_dense_layer['B'+str(i+1)](x1)
            x1_final = tf.add(tf.multiply(w_g, xg_hidden), x1_final)
            
        ## Step 3: Softmax
        logit = tf.reshape(x1_final, (-1, self.k))
        softmax_p =  self.softmax(logit, axis=-1)

        ## Outcome 
        ypredicts = self.outcome(x1)
        ypredicts = tf.squeeze(ypredicts, axis=-1)

        y2 = tf.reduce_sum(tf.multiply(exposure, ypredicts), axis = 1, keepdims=True)
        res = tf.concat([softmax_p, logit, y2, ypredicts], axis=1)
        return res

# Define custom loss function
def custom_loss(y_true, y_pred):
    y1_true, y2_true = tf.split(y_true, [K, 1], axis=1)
    _, y1_logit_pred, y2_pred, _= tf.split(y_pred, [K, K, 1, K], axis=1)
    loss1 = tf.keras.losses.CategoricalCrossentropy(from_logits=True)(y1_true, y1_logit_pred)
    loss2 = tf.keras.losses.MeanSquaredError()(y2_true, y2_pred)
    return loss1 + loss2

In [5]:
## Modifying the tensor for 3d input 
class MyModel_true():
    def __init__(self, k, promo):
        self.k = k
        self.promo = promo
        
    
    def predict(self, inputs):

        X_goodbads, X_utility, W_matrix, exposure_matrix = np.split(inputs, [1,2,3], axis=2)
        logit = self.promo * W_matrix * X_goodbads + X_utility
        logit = np.squeeze(logit, axis=-1)
        softmax_p =  np.exp(logit) / np.sum(np.exp(logit), axis=1, keepdims=True)

        ypredicts = np.squeeze(X_utility, axis=-1)

        exposure = np.squeeze(exposure_matrix, axis=-1)

        y2 = np.sum(exposure * ypredicts, axis = 1, keepdims=True)
        res = np.concatenate([softmax_p, logit, y2, ypredicts], axis=1)
        return res



In [6]:
class MyModel_random():
    
    def predict(self, inputs):

        X_goodbads, X_utility, W_matrix, exposure_matrix = np.split(inputs, [1,2,3], axis=2)
        K = X_utility.shape[1]
        logit = np.ones_like(X_utility)
        logit = np.squeeze(logit, axis=-1)
        softmax_p = np.ones_like(logit) / K
        ypredicts = np.squeeze(X_utility, axis=-1)

        exposure = np.squeeze(exposure_matrix, axis=-1)

        y2 = np.sum(exposure * ypredicts, axis = 1, keepdims=True)
        res = np.concatenate([softmax_p, logit, y2, ypredicts], axis=1)
        return res


In [7]:
J = 30 
K = 5 
Q = 800 
uplift_factor = 1.0
truth_estimate, truth_stderr = find_ate_ground_truth(J, K, Q, uplift_factor)

L = 1

M = 500 ## Number of iterations for Hessian matrix estimation 
n_folds = 3

In [8]:
B = 1
epochs = 400
np.random.seed(int(time.time() * 1e8 % 1e8))
print("Start K = {}, Q = {}, J = {}".format(str(K), str(Q), str(J)))
for b in range(B):
    (X_utility, X_goodbads, query_matrix, utility_score_matrix, 
     treatment_dict, utility_score, good_bad_dict) = generate_environment(J, K, Q, uplift_factor)
    (query_matrix, X_goodbads, X_utility,W_matrix, exposure_matrix, 
     outcome_potential, X_logit) = DGP_new_heterogeneous(J, Q, K, uplift_factor, query_matrix, X_goodbads, 
                                                         X_utility, treat_control_pool = [True, False])
    observed_queries_treatment = np.sum(exposure_matrix * W_matrix, axis = 1 )
    observed_outcome = np.sum(outcome_potential * exposure_matrix, axis = 1 )
    T, C = observed_outcome[observed_queries_treatment == 1] , observed_outcome[observed_queries_treatment == 0]  
        
    
    ## Cross-fitting indices 
    all_inds = generate_indices(np.array(query_matrix).shape[0], n_folds)

    ## Iterate over each fold for cross-validation. 
    hfuncs_each_fold,  debias_terms_each_fold = {}, {}

    for f in range(n_folds):
        f_start, f_end = all_inds[f]
        f_size = f_end - f_start
        
        ## Cross-fitting
        X_goodbads_train, X_goodbads_test =  train_test_split(X_goodbads, all_inds, f) 
        X_utility_train, X_utility_test =  train_test_split(X_utility, all_inds, f)  
        W_matrix_train, W_matrix_test = train_test_split(W_matrix, all_inds, f) 
        exposure_matrix_train, exposure_matrix_test =train_test_split(exposure_matrix, all_inds, f) 
        observed_outcome_train, observed_outcome_test = train_test_split(observed_outcome, all_inds, f) 
        
        outcome_potential_train, outcome_potential_test = train_test_split(outcome_potential, all_inds, f)  
    
        inputs_3d_train = np.stack([X_goodbads_train, X_utility_train, W_matrix_train, exposure_matrix_train], axis = -1)
        inputs_3d_test = np.stack([X_goodbads_test, X_utility_test, W_matrix_test, exposure_matrix_test], axis = -1)
        output_3d_train = np.concatenate([exposure_matrix_train.astype(dtype=float), observed_outcome_train[:, np.newaxis]], axis = 1)

        myModelMultiple = MyModel_multiple(K, 1)
        myModelMultiple.compile(loss=custom_loss, optimizer=tf.keras.optimizers.legacy.Adam())
        myModelMultiple.fit(inputs_3d_train, output_3d_train, epochs=epochs, verbose=True)
        # myModelMultiple = MyModel_true(K, uplift_factor)
        # myModelMultiple = MyModel_random()

        predict_p_test, _, _, predict_outcome_test = np.split(myModelMultiple.predict(inputs_3d_test), [K, 2*K, 2*K+1], axis=1)

        input_3d_test_treat = np.stack([X_goodbads_test, X_utility_test, np.ones_like(W_matrix_test), exposure_matrix_test], axis = -1)
        input_3d_test_control = np.stack([X_goodbads_test, X_utility_test, np.zeros_like(W_matrix_test), exposure_matrix_test], axis = -1)
        
        predict_p_treat, _, _, predict_outcome_treat = np.split(myModelMultiple.predict(input_3d_test_treat), [K, 2*K, 2*K+1], axis=1)
        predict_p_control, _, _, predict_outcome_control = np.split(myModelMultiple.predict(input_3d_test_control), [K, 2*K, 2*K+1], axis=1)
        

        ## 1. COMPUTE THE GRADIENT OF LOSSS  
        gradient_vector_l = compute_loss_gradient(predict_p_test, exposure_matrix_test, W_matrix_test, 
                                                  predict_outcome_test, observed_outcome_test)




        ## 2. COMPUTE  THE GRADIENT OF H FUNCTION
        gradient_vector_H = compute_value_gradient(predict_p_treat, predict_outcome_treat, predict_p_control, predict_outcome_control)

        
        # 3. FIND THE EXPECTATION OF HESSIAN MATRIX 
        Hessian_all = np.zeros((f_size, (L+2) * K - 1,  (L+2) * K - 1))
        for m in range(M):
            treat_dict_m = permute_treatment_dict(J, L)
            W_matrix_m = []
            for each_query in query_matrix[f_start:f_end]:
                W_matrix_m.append([treat_dict_m[ind] for ind in each_query])
            W_matrix_m = np.array(W_matrix_m)
            inputs_m = tf.stack([X_goodbads_test, X_utility_test, W_matrix_m, exposure_matrix_test], axis = -1)
            predict_p_m, _, _, _ = np.split(myModelMultiple.predict(inputs_m), [K, 2*K, 2*K+1], axis=1)
            Hessian = compute_hessian_instance(W_matrix_m, predict_p_m)
            Hessian_all = Hessian_all + Hessian
        Hessian_final = Hessian_all / M
        
        count_finite = 0
        debias_term_f = np.zeros(len(Hessian_final))
        for i in range(f_size):
            if is_invertible(Hessian_final[i]):
                try:
                    debias_term_f[i] = gradient_vector_H[i]@np.linalg.inv(Hessian_final[i])@gradient_vector_l[i]
                    count_finite += 1 
                except: 
                    print("Fail for inversion")


        ## END OF FOR LOOP FOR EACH ITERATION OVER CROSS FITTING
        hfuncs_each_fold[f] = np.sum(predict_p_treat * predict_outcome_treat, axis=1) - np.sum(predict_p_control * predict_outcome_control, axis=1)
        debias_terms_each_fold[f] = debias_term_f
        
    (debias_point, debias_var, undebias_point, undebias_var) = crossfitted_estimate_var(hfuncs_each_fold, debias_terms_each_fold)
    dim_point, dim_var = dim_est(T, C, 0.5, Q)

    path = compose_filename(f"results1106/new_heterogeneous_synthetic_ab_j{J}q{Q}k{K}_100_{uplift_factor}", "csv")
    result_df = pd.DataFrame({"debias_point": [debias_point], "debias_var":[debias_var], "dim": [dim_point], 
                              "dim_var":[dim_var], "undebias_point": [undebias_point], "undebias_var": [undebias_var], 
                              "J" : [J], "Q": [Q],  "K":[K], "truth": [truth_estimate], "truth_stderr": [truth_stderr] })
    result_df.to_csv(path)
    print("finish simulation.")


Start K = 5, Q = 800, J = 30


2024-06-11 14:35:54.127498: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37132 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


Epoch 1/10


2024-06-11 14:35:54.129761: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 37798 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0
2024-06-11 14:35:54.131625: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 28265 MB memory:  -> device: 2, name: A100-SXM4-40GB, pci bus id: 0000:81:00.0, compute capability: 8.0
2024-06-11 14:35:54.133326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 28279 MB memory:  -> device: 3, name: A100-SXM4-40GB, pci bus id: 0000:c1:00.0, compute capability: 8.0


17/17 [==============================] - 2s 4ms/step - loss: 6.1800
Epoch 2/10
17/17 [==============================] - 0s 3ms/step - loss: 5.8274
Epoch 3/10
17/17 [==============================] - 0s 3ms/step - loss: 5.5035
Epoch 4/10
17/17 [==============================] - 0s 3ms/step - loss: 5.1956
Epoch 5/10
17/17 [==============================] - 0s 3ms/step - loss: 4.9354
Epoch 6/10
17/17 [==============================] - 0s 3ms/step - loss: 4.6900
Epoch 7/10
17/17 [==============================] - 0s 3ms/step - loss: 4.4777
Epoch 8/10
17/17 [==============================] - 0s 3ms/step - loss: 4.2779
Epoch 9/10
17/17 [==============================] - 0s 3ms/step - loss: 4.1042
Epoch 10/10
9/9 [==============================] - 0s 2ms/step
Epoch 1/10
17/17 [==============================] - 0s 3ms/step - loss: 80.1444
Epoch 2/10
17/17 [==============================] - 0s 3ms/step - loss: 78.1997
Epoch 3/10
17/17 [==============================] - 0s 3ms/step - loss: 76.16

In [10]:
result_df

,debias_point,debias_var,dim,dim_var,undebias_point,undebias_var,J,Q,K,truth,truth_stderr
0,0.061009,133.579209,0.813835,116.689613,-0.802028,3.132969,30,800,5,0.017009,0.000638
